TODO: I need to fix this because WhisperX has a bad Pyannote version

# Step 0: Preprocess data

- `.wav` format
- trimmed to your desire segment

# Step 1: Transcribing an audio file

In [1]:
from dotenv import load_dotenv
import os

load_dotenv()
hf_key = os.getenv("HF_KEY")

In [ ]:
!pip install whisperx
!pip install torch

In [ ]:
# If you want to use a GPU for faster transcriptions (e.g. if you have Colab kernel connected)
import torch

if torch.cuda.is_available():
    print("CUDA is available! Using GPU.")
    device = torch.device("cuda")
    # You can also get more information about the GPU
    print(f"Number of GPUs: {torch.cuda.device_count()}")
    for i in range(torch.cuda.device_count()):
        print(f"GPU {i}: {torch.cuda.get_device_name(i)}")
else:
    print("CUDA is not available. Falling back to CPU.")
    device = "cpu"

print(f"Current device: {device}")

CUDA is not available. Falling back to CPU.
Current device: cpu


In [ ]:
import whisperx
import gc

audio_file = 'input/DINA1_PS1_IDS1.wav'
batch_size = 16
compute_type = "float32"
model_name = "medium" # Empirically I've seen that medium performs best in getting the utterance timings!

# 1. Transcribe with original whisper (batched) 13 mins on CPU
model = whisperx.load_model(model_name, device, compute_type=compute_type)
audio = whisperx.load_audio(audio_file)
result = model.transcribe(audio, batch_size=batch_size, language='es') # NOTE: For some reason on code-switching audio it's best to set it to the less dominant language
print(result["segments"]) # before alignment

In [16]:
import json
import os

# Save raw whisper transcription to disk
basename = os.path.basename(audio_file).split('.')[0]
with open(f"output/{basename}_WhisperX_Medium_raw.json", "w") as f:
    json.dump(result, f, indent=4)

# Step 2: Alignment

In [ ]:
# 2. Align whisper output 4 mins on CPU
model_a, metadata = whisperx.load_align_model(language_code=result["language"], device=device)
result = whisperx.align(result["segments"], model_a, metadata, audio, device, return_char_alignments=False)

print(result["segments"]) # after alignment

In [ ]:
# 3. Assign speaker labels 52 mins on CPU
diarize_model = whisperx.diarize.DiarizationPipeline(use_auth_token=HF_TOKEN, device=device)

# add min/max number of speakers if known
diarize_segments = diarize_model(audio)
diarize_model(audio, min_speakers=1, max_speakers=4)

result = whisperx.assign_word_speakers(diarize_segments, result)
print(diarize_segments)
print(result["segments"]) # segments are now assigned speaker IDs

In [ ]:
import json
with open("output/dina1_playsesh_whisperX_medium_final.json", "w") as f:
    json.dump(result, f, indent=4)

In [ ]:
import json

with open("output/dina1_playsesh_whisperX_medium_final.json", "r") as f:
    result = json.load(f)


In [ ]:
# Final results
print(result['segments'])
print(result['word_segments'])